In [1]:
using StarStats

Load the entire database of grids 

In [2]:
using Printf
using BenchmarkTools
function path_constructor(strings::Vector{String})
    DATA_FOLDER = ENV["STARSTATS_TEST_DATA_FOLDER"]
    return DATA_FOLDER*"/LMC/LMC_$(strings[1])_$(strings[2])_$(strings[3]).track.gz"
end
masses = [@sprintf("%.3f", x) for x in range(0.9,2.1,step=0.025)]
rotation = [@sprintf("%.2f", x) for x in range(0.0,0.9,step=0.1)]
overshoot = [@sprintf("%.2f", x) for x in range(0.5,4.5,step=0.5)]
grid = ModelDataGrid([rotation,masses,overshoot],
    [:rotation,:logM,:overshoot])
#@benchmark 
load_grid(grid,path_constructor,gz_dataframe_loader_with_Teff_and_star_age_fix); 
compute_distances_and_EEPs(grid)

In [3]:
observable_names = [:logTeff, :logL, :vrot]
observable_values = [4.51974, 4.289877, 70.7195]
observable_errors = [0.2, 0.2, 10]



3-element Vector{Float64}:
  0.2
  0.2
 10.0

In [108]:
using Turing, Distributions

@model function star_model(logTeff_obs, logTeff_err, logL_obs, logL_err, vrot_obs, vrot_err, grid)
  x ~ Uniform(0,3)
  logM ~ Uniform(0.9, 1.5)
  rotation ~ Uniform(0,0.9)
  overshoot ~ Uniform(0.5,1.5)
  logTeff = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:logTeff,x)
  logL = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:logL,x)
  vrot = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:vrot,x)
  

  logTeff_obs ~ Normal(logTeff, logTeff_err)
  logL_obs ~ Normal(logL, logL_err)
  vrot_obs ~ Normal(vrot, vrot_err)
  return logTeff_obs, logL_obs, vrot_obs
end

star_model (generic function with 2 methods)

In [109]:
using Logging
Logging.disable_logging(Logging.Warn)
## Here needs more analysis for optimization
num_chains=4

observed_star_model = star_model(4.51974, 0.2, 4.289877, 0.2, 70.7195, 20, grid)
star_chains = mapreduce(c -> sample(observed_star_model, NUTS(500,0.9), 20000;stream=false, progress=true), chainscat, 1:num_chains)

Chains MCMC chain (20000×16×4 Array{Float64, 3}):

Iterations        = 501:1:20500
Number of chains  = 4
Samples per chain = 20000
Wall duration     = 35.79 seconds
Compute duration  = 34.99 seconds
parameters        = x, logM, rotation, overshoot
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse          ess      rhat   ⋯
      Symbol   Float64   Float64    Float64   Float64      Float64   Float64   ⋯

           x    1.0454    0.6324     0.0022    0.0055   15032.1002    1.0002   ⋯
        logM    1.0864    0.0834     0.0003    0.0006   23204.1033    1.0001   ⋯
    rotation    0.1223    0.0494     0.0002    0.0007    5255.3075    1.0012   ⋯
   overshoot    0.9944    0.2882     0.0010    0.0014   37383.5831    1.0000   ⋯
                                         

In [ ]:
histogram(star_chains[:x],alpha=0.5,nbins=30)

In [119]:
#logM = star_chains[
 
logM = star_chains[:,:logM, :1].data
append!(logM,star_chains[:,:logM, :2].data)
append!(logM,star_chains[:,:logM, :3].data)
append!(logM,star_chains[:,:logM, :4].data)
@show length(logM)

rotation = star_chains[:,:rotation, :1].data
append!(rotation,star_chains[:,:rotation, :2].data)
append!(rotation,star_chains[:,:rotation, :3].data)
append!(rotation,star_chains[:,:rotation, :4].data)

overshoot= star_chains[:,:overshoot, :1].data
append!(overshoot,star_chains[:,:overshoot, :2].data)
append!(overshoot,star_chains[:,:overshoot, :3].data)
append!(overshoot,star_chains[:,:overshoot, :4].data)

x= star_chains[:,:x, :1].data
append!(x,star_chains[:,:x, :2].data)
append!(x,star_chains[:,:x, :3].data)
append!(x,star_chains[:,:x, :4].data)


##problem: how to get dtdx --- below is messed up 
dtdx=zeros(length(logM))
for i in 1:length(logM)
    dtdx[i]= interpolate_grid_quantity(grid,[0.13,1.21,1.05],:dxdt,x[i])
    #interpolate_grid_quantity.(Ref(grid),Ref([0.13,1.21,1.05]),:logTeff, xvals)
end


In [79]:
using QuadGK
function chain_credible_interval(values, fraction,num_points,weights)
    maxval= maximum(logM)
    minval = minimum(logM)
    my_kde = kde(logM, weights=weights)
    kde_function = x-> pdf(my_kde, x)
    
    integral = quadgk(kde_function,minval,maxval, rtol=1e-8)[1]
    
    xvalues = collect(LinRange(minval,maxval,num_points))
    yvalues = kde_function(xvalues)
    max_valy = findmax(yvalues)
    maxx = xvalues[max_valy[2]]
    maxy = max_valy[1]
    
    function kde_function2(x, kde_function, bound)
        value = kde_function(x)
        if value>bound
            return value
        else
            return 0
        end
    end
    minbound = 0
    maxbound = maxy
    newbound = 0
    for i in 1:15
        newbound = 0.5*(minbound+maxbound)
        integral2 = quadgk(x -> kde_function2(x,kde_function,newbound), minval, maxval, rtol=1e-8)[1]
        newfraction = integral2/integral
        if newfraction>fraction
            minbound = newbound
        else
            maxbound = newbound
        end
    end
    filtered_xvalues = xvalues[yvalues.>newbound]
    return ([maximum(filtered_xvalues), maxx, minimum(filtered_xvalues)], newbound)
end

chain_credible_interval (generic function with 1 method)

In [120]:
num_points = 10000
values = logM
fraction = 0.9


#CI = chain_credible_interval(values, fraction, num_points,dtdx)
@show dtdx

80000-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [83]:
observable_names = [:logTeff, :logL, :vrot]
observable_values = [4.51974, 4.289877, 70.7195]
observable_errors = [0.2, 0.2, 20]
# potentially parallelize this
grid_likelihood = ModelDataGridLikelihood(grid, observable_names, observable_values, observable_errors);
CI2 = credible_interval(grid_likelihood, :logM,0.9,1_000_000)

([1.2283107283107284, 1.125000225000225, 0.9642552642552642], 4192.989242514728)

In [84]:
using Plots
plot()
ml= marginalized_likelihood(grid_likelihood,[:logM])
plot!(grid.input_values[2], ml./maximum(ml))
hline!([CI2[2]/maximum(ml)])
vline!(CI2[1])

plot!(x->pdf(my_kde,x)/pdf(my_kde,CI[1][2]), xrange=[0.8, 2.1])

hline!([CI[2]/pdf(my_kde,CI[1][2])])
vline!(CI[1])

UndefVarError: UndefVarError: my_kde not defined

To do :

1. MCMC code into its own  file within package
    - make sampling code work with arbitrary observables
    - make credible interval code flexible 
    - optimize the credible interval code 
    - create  pretty corner plots 
    - 
2. grid computed 
3. create simple interpolation interface 
4. change longM with values in credible interval bla



In [88]:
using StatsPlots
using HCubature
using Interpolations
using KernelDensity
#histogram2d(logM, rotation)
my_kde2d = kde((rotation,logM,),weights=dtdx)
kde_function2d = x-> pdf(my_kde2d, x[1],x[2])
#kde_function2d = (x,y)-> pdf(my_kde2d, x,y)

#contourf(my_2dkde)

values1= rotation

values2=logM
maxval1 = maximum(values1)
minval1 = minimum(values1)
maxval2 = maximum(values2)
minval2 = minimum(values2)

a = (minval1, minval2)
b = (maxval1, maxval2)

#integral = hcubature(kde_function2d, a, b)
#@show integral

@show a 
@show b

TypeError: TypeError: in keyword argument weights, expected Union{KernelDensity.UniformWeights, AbstractVector{T} where T<:Real}, got a value of type Symbol

In [ ]:
hcubature(x -> cos(x[1])*cos(x[2]), [0,0], [1,1])[1] 

In [ ]:
using QuadGK
using HCubature
function chain_credible_interval2d(values1,values2, fraction,num_points,weights)
    maxval1 = maximum(values1)
    minval1 = minimum(values1)
    maxval2 = maximum(values2)
    minval2 = minimum(values2)

    my_kde2d = kde((values1,values2),weights=dtdx)
    kde_function2d = (x,y)-> pdf(my_kde2d, x,y)
    
    a = [minval1, minval2]
    b = [maxval1, maxval2]
    integral = hcubature(my_kde2d, a, b; norm=norm, rtol=1e-8, atol=0, maxevals=typemax(Int), initdiv=1)[1]
    quadgk(kde_function2d,minval,maxval, rtol=1e-8)[1]
    
    xvalues = collect(LinRange(minval,maxval,num_points))
    yvalues = kde_function(xvalues)
    max_valy = findmax(yvalues)
    maxx = xvalues[max_valy[2]]
    maxy = max_valy[1]
    
    function kde_function2(x, kde_function, bound)
        value = kde_function(x)
        if value>bound
            return value
        else
            return 0
        end
    end
    minbound = 0
    maxbound = maxy
    newbound = 0
    for i in 1:15
        newbound = 0.5*(minbound+maxbound)
        integral2 = quadgk(x -> kde_function2(x,kde_function,newbound), minval, maxval, rtol=1e-8)[1]
        newfraction = integral2/integral
        if newfraction>fraction
            minbound = newbound
        else
            maxbound = newbound
        end
    end
    filtered_xvalues = xvalues[yvalues.>newbound]
    return ([maximum(filtered_xvalues), maxx, minimum(filtered_xvalues)], newbound)
end